In [ ]:
import numpy as np
import os
import dill as pickle
import matplotlib.pyplot as plt
import pandas as pd
import sys
import hardware_control.wx_programs as wx
import hardware_control.bnc as bnc
from classes.generator import *
from hardware_control.hardware_config import *
from experiment_configuration.values import *
from classes.qubit_class import *
import daq.daq_programs_homo as daq
import standard_sequences.pinopi as pnp
import standard_sequences.sweeppiamp as sweeppiamp

import analysis.analysis as analysis
import seaborn as sns
from scipy.optimize import curve_fit
from skopt import gp_minimize
from skopt.space import Real

In [2]:
q1 = Qubit(q1_dict, readout_dict)
q2 = Qubit(q2_dict, readout_dict)
readout = Readout(readout_dict)
print(f"{q1}\n{q2}")

wx_addr = wx.get_wx_address()
main_directory = r"C:\Users\quantum1\Documents"
save_dir = rf"{main_directory}\Python Scripts\Important Blue Fridge Python Files\New\nonlinear_QM\data"
target_bnc_address_6 = "USB0::0x03EB::0xAFFF::411-433500000-0753::INSTR"
bnc.set_bnc_output(
    general_vals_dict["qubit_bnc"], power_dBm=13, bnc_addr=target_bnc_address_6
)
bnc.set_bnc_output(
    readout_dict["RO_LO"],
    power_dBm=readout_dict["RO_LO_pwr"],
    bnc_addr=bnc_address["target_bnc_black"],
)
bnc.set_bnc_output(
    general_vals_dict["TWPA_freq"],
    general_vals_dict["TWPA_pwr"],
    bnc_addr=bnc_address["big_agilent"],
)


Qubit(RO_LO_pwr=16, ef_amp=1.52, qubit_id=q1, RO_LO=6.6247, ro_freq=6.72739, ge_amp=1.01, IQ_angle=110, ro_dur=5000, RO_IF=None, ge_ssm=-0.1144, qubit_thr=[-10000, -600], ROIF=0.10268999999999995, ef_ssm=-0.2568, ro_amp=0.26, ef_half_time=22, ef_time=43, ef_half_amp=1.54, mixer_offset_ge=2.5, ge_time=78, mixer_offset_ef=6)
Qubit(RO_LO_pwr=16, ef_amp=1, qubit_id=q2, RO_LO=6.6247, ro_freq=6.65554, ge_amp=0.8, IQ_angle=25, ro_dur=5000, RO_IF=None, ge_ssm=-0.154, qubit_thr=[-10000, 1900], ROIF=0.030840000000000423, ef_ssm=-0.2962, ro_amp=0.5, ef_half_time=None, ef_time=None, ef_half_amp=None, mixer_offset_ge=None, ge_time=74, mixer_offset_ef=None)


In [ ]:
def pi_ge_tune(
    q1: object,
    q2: object,
    general_vals_dict: dict,
    num_steps: int,
    reps: int,
    amp: float,
    pi_ge_time: int,
):
    """
    Runs a single instance of the nonhermitian ef Rabi experiment (with an e-swap to Q2)
    and processes the resulting IQ data to compute probabilities vs. time.

    Returns:
        df_prob (pd.DataFrame): A DataFrame whose index is the time (computed as
                                np.linspace(0, sweep_time/1000, num_steps)) and which has
                                columns ['P_f', 'P_e', 'P_g'] corresponding to the probabilities
                                of the f, e, and g states respectively.
    """
    # Run the experiment
    # sweeppiamp.pi_ge_amp(
    #     q1, q2, general_vals_dict, num_steps, amp, pi_ge, swap_freq, swap_time
    # )
    sweeppiamp.pi_ge_gaussian_tune_iq(
    q1,
    q2,
    general_vals_dict,
    num_steps=3,
    amp=0.8,
    pi_ge_time=24,
    swap_freq=-0.21,
    swap_time=213.58765318403013,
    file_length=16000,
    verbose=True,
    )
    wx.wx_set_and_amplitude_and_offset(
        amp=general_vals_dict["wx_amps"], offset=general_vals_dict["wx_offs"]
    )
    # Acquire the raw IQ data
    values = daq.run_daq_het_2q(
        q1, q2, num_patterns=num_steps, num_records_per_pattern=reps, verbose=False
    )

    # Retrieve raw IQ data from the acquired values
    I1_raw = values.rec_readout_1[0]
    Q1_raw = values.rec_readout_1[1]
    I2_raw = values.rec_readout_2[0]
    Q2_raw = values.rec_readout_2[1]

    I1 = np.mean(I1_raw)
    Q1 = np.mean(Q1_raw)
    I2 = np.mean(I2_raw)
    Q2 = np.mean(Q2_raw)

    return I1, Q1, I2, Q2



def get_ground_IQ(reps):
    num_steps = 3
    pnp.pi_nopi_ge(0, 0, q1, q2, general_vals_dict)
    wx.wx_set_and_amplitude_and_offset(
        amp=general_vals_dict["wx_amps"], offset=general_vals_dict["wx_offs"]
    )
    values = daq.run_daq_het_2q(
        q1, q2, num_patterns=num_steps, num_records_per_pattern=reps, verbose=True
    )

    # Retrieve raw IQ data from the acquired values
    I1_raw = values.rec_readout_1[0][0 : reps * num_steps]
    Q1_raw = values.rec_readout_1[1][0 : reps * num_steps]
    I2_raw = values.rec_readout_2[0][0 : reps * num_steps]
    Q2_raw = values.rec_readout_2[1][0 : reps * num_steps]

    I1 = np.mean(I1_raw)
    Q1 = np.mean(Q1_raw)
    I2 = np.mean(I2_raw)
    Q2 = np.mean(Q2_raw)

    return I1, Q1, I2, Q2

In [ ]:
space = [Real(0.1,1.5, name= 'amp'),Real(15,40, name='pi_ge_time')]
reps = 50000
I1_ground, Q1_ground, I2_ground, Q2_ground = get_ground_IQ(reps)

def minimization_function(params):

    amp, pi_ge_time = params
    I1, Q1, I2, Q2 = pi_ge_tune(
    q1,
    q2,
    general_vals_dict,
    num_steps = 3,
    reps= reps,
    amp=amp,
    pi_ge_time=pi_ge_time)

    # Compute total L2 distance from both qubit IQ references
    err_q1 = (I1_ground - I1)**2 + (Q1_ground - Q1)**2
    err_q2 = (I2_ground - I2)**2 + (Q2_ground - Q2)**2

    return err_q1 + err_q2 


result = gp_minimize(
    func=minimization_function,
    dimensions=space,
    acq_func="EI",            
    n_calls=50,                
    n_initial_points=10,        
    noise="gaussian",          
    random_state=42            
)

print("Optimal amplitude:", result.x[0])
print("Optimal pi_ge_time:", result.x[1])
print("Minimum total IQ error:", result.fun)


[Real(low=0.1, high=1.5, prior='uniform', transform='identity'),
 Real(low=15, high=40, prior='uniform', transform='identity')]